Le assunzioni che faccio è che la prima stringa sia fatta dalla generale
la seconda stringa indica come si calcola il primo elemento e la terza indica come si calcola il secondo elemento e cosi via e poi se gli elementi coinvolti sono coinvolti nel primo allora si fa quello. la cosa deve essere ricorsiva. Quindi parsata la stringa devo andare a controllare la stringa dopo e cosi via. Possiamo fare una logica ricorsiva

In [8]:
#proviamo il parsing delle stringhe
import re
from collections import defaultdict
stringa="A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]"
stringa1="S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"
stringa3="A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]"

#ho bisogno di isolare i nomi dei kpi le operazioni, tempo e machine li abbiamo dal RAG
#ho bidogno di altre cose da fare tipo andare a identificare le lettere tipo A e B

#trovare la logica migliore per il parsing di queste 3 stringhe
#pattern = r"([A-Z])°([\w]+)°([\w]+)\[|\[|\]|;|([A-Z])°([\w]+)°([\w]+)"

#patter per dividere tutto ciò che trova nella stringa, ma come si fa a dare significato ad ogni punto?
#token_pattern = r"[A-Z]+|[a-zA-Z0-9_]+|[\[\]/;+\-*/=]"

#questo cerco di dividere solo quello dentro le parentesi quadre funziona?? può essere utile
pattern = r"([A-Z])°([a-zA-Z0-9__+\-*/=]+)"

#posso fare questa assunzione? sono mappate uno a uno con le macchine
pattern_oper = r"T°M°([a-zA-Z0-9_]+)"

#altro possibile pattern
pattern_pp = r"([a-zA-Z0-9_]+)°([a-zA-Z0-9_]+)(?:°([a-zA-Z0-9_]+))?"

#find all ritorna una lista che poi è da parsare
a=set(re.findall(r"\b[A-Za-z][A-Za-z0-9]*\b", stringa))

#posso affrontare il problema in diversi modi
b=re.findall(pattern, stringa)
#faccio il parsing delle stringhe e vediamo

'''
dizionario=defaultdict(list)

for key, value in b:
    dizionario[key].append(value)

print(dict(dizionario))
'''
b

[('A', 'mean'),
 ('S', '/'),
 ('R', 'consumption_sum'),
 ('T', 'm'),
 ('R', 'time_sum'),
 ('T', 'm')]

Abbiamo ottenuto una lista per ogni lettera e delle variabili e les tringhe sono parsate. Adesso devo stabilire una logica

In [ ]:
import re
from collections import defaultdict
from datetime import datetime

#Mi arrivano queste tre stringhe
stringa="A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]"
stringa1="S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"
stringa3="A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]"

# a me arriva questo che è la formula
formule=["A°mean°mo[ S°/[ R°consumption_sum°T°m°o° ; R°time_sum°T°m°o° ]]",
         "A°sum°mo[ A°sum°t[ D°consumption_sum°t°m°o° ] ]",
         "S°+[ R°time_sum°T°M°idle° ; R°time_sum°T°M°offline° ]"]

#io ho già informazioni del tipo

step=7
macchine=['ma','mb']
operazioni=['idle','offline']
start_date=datetime(2023, 1, 1),
end_date=datetime(2023, 12, 31),
aggregazione='mean'

############################

#Logica di parsing delle stringhe
pattern1 = r"([a-zA-Z0-9_/\+\-\*\=]+(?:°[a-zA-Z0-9_/\+\-\*\=]+)*)"
# Troviamo tutti i match principali
matches = re.findall(pattern1, stringa)
# Dividiamo ogni match in tuple
parsed_results = [tuple(item.split("°")) for item in matches]
# Risultato finale
print(parsed_results)

[('A', 'mean', 'mo'), ('S', '/'), ('R', 'consumption_sum', 'T', 'm', 'o'), ('R', 'time_sum', 'T', 'm', 'o')]


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
a=pd.DataFrame(data = [
    {"KPI": "consumption_sum", "Operazione": 'idle', "macchina": 'A', "Date": datetime(2023,10,10),'vale':30},
    {"KPI": "consumption_sum", "Operazione": 'working', "macchina": 'B', "Date": datetime(2023,10,11),'vale':20},
    {"KPI": "consumption_sum", "Operazione": 'idle', "macchina": 'A', "Date": datetime(2023,10,12),'vale':10},
])


a=a.drop(columns=['KPI'])
#print(a)
pivot_table = a.pivot(
            index="Date", columns=["macchina",'Operazione'], values="vale"
        ).reset_index()


pivot_table


macchina,Date,A,B
Operazione,,idle,working
0,2023-10-10,30.0,NaN
1,2023-10-11,NaN,20.0
2,2023-10-12,10.0,NaN


In [44]:
a=pd.DataFrame(data = [
    { "C": 'A_idle',  "Date": datetime(2023,10,10),'max':30},
    { "C": 'B_working', "Date": datetime(2023,10,11),'max':20},
    { "C": 'C_idle', "Date": datetime(2023,10,12),'max':10},
    { "C": 'B_working', "Date": datetime(2023,10,12),'max':10},
])

pivot_table = a.pivot(
            index="Date", columns=['C'], values="max"
        ).reset_index().select_dtypes('number')

pivot_table2=a.pivot(
            index="Date", columns=['C'], values="max"
        ).reset_index().select_dtypes('number')

#pivot_table

c=pivot_table+pivot_table2
c

C,A_idle,B_working,C_idle
0,60.0,NaN,NaN
1,NaN,40.0,NaN
2,NaN,20.0,20.0


In [148]:


a=np.arange(60).reshape(10, 6)

a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41],
       [42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59]])

In [5]:
a=np.arange(60).reshape(10, 6)
step=3
resto=a.shape[0]% step

fondo=a[-resto:]
a = a[:-resto]
#g=a.reshape(resto,step,a.shape[1])
groups = a.reshape(a.shape[0] // step, step, a.shape[1])
n=[groups,fondo]
print(groups)
print('-----------------------')
n[0]=np.sum(n[0],axis=1)
print(n[0])
print('-----------------------')

n[0]=np.sum(n[0],axis=1)
print(n[0])



[[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]
  [12 13 14 15 16 17]]

 [[18 19 20 21 22 23]
  [24 25 26 27 28 29]
  [30 31 32 33 34 35]]

 [[36 37 38 39 40 41]
  [42 43 44 45 46 47]
  [48 49 50 51 52 53]]]
-----------------------
[[ 18  21  24  27  30  33]
 [ 72  75  78  81  84  87]
 [126 129 132 135 138 141]]
-----------------------
[153 477 801]


In [ ]:
s="A°mean°mo[S°time]"
a=s.split('°')

['A', 'mean', 'mo[S', 'time]']

In [26]:
import pandas as pd
import numpy as np

date_range = pd.date_range(start="2023-11-10", end="2023-12-10", freq="D")

# Generazione di valori casuali per le due colonne
np.random.seed(42)  # Per rendere i risultati ripetibili
cut_machine_idle = np.random.uniform(0, 100, len(date_range))
cut_machine_forward = np.random.uniform(0, 100, len(date_range))

# Creazione del DataFrame
dataframe = pd.DataFrame({
    "date": date_range,
    'kpi': 'A',
    "cut_machine_idle": cut_machine_idle,
    "cut_machine_forward": cut_machine_forward
})

a=np.array_split(dataframe, 5)
a

[        date kpi  cut_machine_idle  cut_machine_forward
 0 2023-11-10   A         37.454012            17.052412
 1 2023-11-11   A         95.071431             6.505159
 2 2023-11-12   A         73.199394            94.888554
 3 2023-11-13   A         59.865848            96.563203
 4 2023-11-14   A         15.601864            80.839735
 5 2023-11-15   A         15.599452            30.461377
 6 2023-11-16   A          5.808361             9.767211,
          date kpi  cut_machine_idle  cut_machine_forward
 7  2023-11-17   A         86.617615            68.423303
 8  2023-11-18   A         60.111501            44.015249
 9  2023-11-19   A         70.807258            12.203823
 10 2023-11-20   A          2.058449            49.517691
 11 2023-11-21   A         96.990985             3.438852
 12 2023-11-22   A         83.244264            90.932040,
          date kpi  cut_machine_idle  cut_machine_forward
 13 2023-11-23   A         21.233911            25.877998
 14 2023-11-24   A  